In [526]:
import sys
import os
import urllib
sys.path.append("..")
#from utils.case import Case, DocketEntry, Document, Party
import pandas as pd
import csv

DATA_LOCATION = "../data/amazon-textract-textractor-master"

table_files = []
total_explored = 0
for file in os.listdir(DATA_LOCATION):
    if file.endswith("tables.csv"):
        total_explored += 1
        filename = os.path.join(DATA_LOCATION, file)
        empty = True
        with open(filename,'rt')as f:
            data = csv.reader(f)
            for row in data:
                #print(row)
                empty = False
                break
        #df = pd.read_csv(filename)
        if not empty:
            table_files.append(file)
print("done")
print("total explored: ", total_explored)
print("total kept: ", len(table_files))

KeyboardInterrupt: 

In [ ]:
#extra imports
import string
import numpy as np
import re
import pprint

In [418]:
def getNumsFromString(item):
    nums = []
    num = ''
    wordAfterNum = ''
    dig = False
    after = False
    for c in item:
        if (c.isdigit()):
            dig = True
            num += c
        elif c == ',' and dig == True:
            continue
        elif c == '.' and dig == True:
            num += c
        elif dig == True:
            try:
                nums.append(float(num))
            except:
                pass
            num = ''
            dig = False
            after = True
        elif after == True:
            if c.isalpha():
                wordAfterNum += c
            else:
                after = False
                wordAfterNum.replace(" ", "")
                
                if wordAfterNum.lower() == "billion" or wordAfterNum.lower() == "b":
                    nums[-1] *= 1000000000
                if wordAfterNum.lower() == "million" or wordAfterNum.lower() == "m":
                    nums[-1] *= 1000000
                if wordAfterNum.lower() == "thousand":
                    nums[-1] *= 1000
                wordAfterNum = ''
   
    return nums

In [419]:
def checkForNumbered(creditors):
    nums = []
    #print("hello from checkfornumbered")
    for s in creditors:
        words = s.split()
        word1 = words[0].replace('.', '')
        #print(word1)
        if word1.isdigit():
            num = int(word1)
            if num < 50:
                nums.append(num)
    numbered = True
    toRemove = []
    if len(nums) > 1:
        prevNum = nums[0]
        for i in range(1, len(nums)):
            if nums[i] == prevNum + 1:
                prevNum = nums[i]
            else:
                toRemove.append(nums[i])
                
        for num in toRemove:
            nums.remove(num)
        #print(nums)
        if len(nums) < 2:
            numbered = False
            
    else:
        numbered = False
    if numbered: #remove numbers
        #print("numbered")
        for i, s in enumerate(creditors):
            words = s.split()
            word1 = words[0].replace('.', '')
            if word1.isdigit():
                num = int(word1)
                if num in nums:
                    words.pop(0)
                    nums.remove(num)
            s = ' '.join(words)
            creditors[i] = s
        

In [469]:
acctNo = ["account no", "acecunt no", "acct #", "acct no", "accoumnt no", 
         " account no", " acecunt no", " acct #", " acct no", " accoumnt no", "account number"]
creditNo = ["creditor #", "credit tor #", "credi tor #", "credito #", "credito: #",
           "creditor", "credit tor", "credi tor", "credito"]

In [460]:
def checkForAcctNo(creditors):
    for i, creditor in enumerate(creditors):
        for s in acctNo:
            if creditor.lower().startswith(s):
                creditor = creditor[len(s):]
                numToRemove = 0
                for c in creditor:
                    if c == ' ' or c in string.punctuation:
                        numToRemove += 1
                    else:
                        break
                creditor = creditor[numToRemove:]
                if len(creditor) > 0:
                    if creditor[0].isdigit():
                        words = creditor.split()
                        words.pop(0)
                        creditor = ' '.join(words)


        creditors[i] = creditor
                


In [454]:
l = ["ACCOUNT NO.", "bleh"]
checkForAcctNo(l)
print(l)

['bleh']


In [461]:
def checkForCreditorNo(creditors):
    for i, creditor in enumerate(creditors):
        for s in creditNo:
            if creditor.lower().startswith(s):
                creditor = creditor[len(s):]
                numToRemove = 0
                for c in creditor:
                    if c == ' ' or c in string.punctuation:
                        numToRemove += 1
                    else:
                        break
                creditor = creditor[numToRemove:]

        creditors[i] = creditor

In [426]:
creditors_test = ["Credi tor # : 22 Godfrey - Son Auto 2354 Gleason Avenue Bronx NY 10462"]
print(creditors_test)
checkForCreditorNo(creditors_test)
print(creditors_test)

['Credi tor # : 22 Godfrey - Son Auto 2354 Gleason Avenue Bronx NY 10462']
['22 Godfrey - Son Auto 2354 Gleason Avenue Bronx NY 10462']


In [424]:
not204words = ["wearing apparel", "estimated", "cash on hand", "checking, savings", "b -", "c -", "d -", "e -", "f -", "g -", "h -","i -", "j -",
              "student loan obligations", "farming equipment", "itemize"]
wordsToDrop = ["nan", "continuation sheets", "sheet no", "estimated", "total", 
              "name of creditor", "and complete", "mailing address including zip code", "unsecured nonpriority claims"]

In [427]:
def toRemove(creditors):
    removing = []
    for creditor in creditors:
        for word in not204words:
            if word in creditor.lower():
                #print("returning false on word: ", word)
                return False #not form 204
        for word in wordsToDrop:
            if word in creditor.lower():
                #print("removing: ", word)
                removing.append(creditor)
    for creditor in removing:
        try:
            creditors.remove(creditor)
        except Exception:
            #if something was to be removed more than once, the E - 's in particular
            pass
        
    return True

In [428]:
poBoxWords = ["po box", "p.o. box", "p o box", "p. o. box"]
numberWords = ["one", "two", "three", "four", "five", "six", "seven", "eight", "nine",
           "eleven", ]
addrIndic = ["way", "plaza", "broadway", "building", "street", "st", "drive", "dr", 
            "place", "pl", "boulevard", "blvd", "rd", "road", "square", "sq", "center", "ctr",
            "ave", "avenue", "suite", "lane", "ln"]

In [462]:
def cutOffAddresses(creditors):
    for creditIndex, creditor in enumerate(creditors):
        addressFound = False
        for word in poBoxWords:
            index = creditor.lower().find(word)
            if index != -1:
                creditor = creditor[:index]
        words = creditor.split()
        toremove = -1
        numWords = len(words)
        for i, word in enumerate(words):
            word = word.lower()
            word = word.translate(str.maketrans('', '', string.punctuation))
            if(word.isdigit()):
                toremove = i
                break
            elif word in numberWords:
                addrFound = False
                for j in range(1, 4):
                    if i + j >= numWords:
                        break
                    else:
                        if words[i + j].lower() in addrIndic:
                            
                            addrFound = True
                            toremove = i
                            break
                if addrFound:
                    break
        if toremove > -1:
            words = words[:toremove]
        creditor = ' '.join(words)
        creditors[creditIndex] = creditor
    
        

In [507]:
states = ['alabama', 'alaska', 'american samoa', 'arizona', 'arkansas', 'california', 'colorado', 'connecticut', 'delaware', 'district of columbia', 'florida', 'georgia', 'guam', 'hawaii', 'idaho', 'illinois', 'indiana', 'iowa', 'kansas', 'kentucky', 'louisiana', 'maine', 'maryland', 'massachusetts', 'michigan', 'minnesota', 'minor outlying islands', 'mississippi', 'missouri', 'montana', 'nebraska', 'nevada', 'new hampshire', 'new jersey', 'new mexico', 'new york', 'north carolina', 'north dakota', 'northern mariana islands', 'ohio', 'oklahoma', 'oregon', 'pennsylvania', 'puerto rico', 'rhode island', 'south carolina', 'south dakota', 'tennessee', 'texas', 'u.s. virgin islands', 'utah', 'vermont', 'virginia', 'washington', 'west virginia', 'wisconsin', 'wyoming']

In [509]:
def cityStateZip(creditors):
    for creditIndex, creditor in enumerate(creditors):
        words = creditor.split()
        if len(words) != 0:
            lastWord = words[-1]
            z = re.match("^\d{5}(-\d{4})?$", lastWord)
            if z and len(words) < 7:
                #print("deleting creditor: ", creditor)
                creditor = ""
            else:
                t = re.match("^[A-Z][A-Z]$", lastWord)
                stateFound = False
                if t:
                    stateFound = True
                else:
                    if lastWord.lower() in states:
                        stateFound = True
                #print("matched: ", lastWord)
                if len(words) > 1:
                    secondToLast = words[-2]
                    #print(secondToLast)
                    #print("," in secondToLast)
                    #print(t)
                    if stateFound and ("," in secondToLast):
                        if (len(words) < 5):
                            #print("deleting")
                            creditor = ""
            creditors[creditIndex] = creditor


        

In [503]:
l = ["Montgomery, NY"]
cityStateZip(l)
print(l)

['']


In [466]:
def remBlanks(creditors):
    while("" in creditors) : 
        creditors.remove("")

In [467]:
import string

def cleanupCreditors(creditors):
    if not toRemove(creditors):
        return False
    checkForAcctNo(creditors)
    remBlanks(creditors)
    checkForNumbered(creditors) #will remove numbering if it exists
    cutOffAddresses(creditors)
    remBlanks(creditors)
    cityStateZip(creditors)
    remBlanks(creditors)
    return True

In [527]:
i = 0
#amountDict = {}
forms = {}
for file in table_files:
    #print(file)
    creditors = []
    amounts = []
    caseID = file.split('_')[0]
    filename = os.path.join(DATA_LOCATION, file)
    #if i > 0:
        #break
    try:
        df = pd.read_csv(filename, skiprows=2) #skip over column headers too, since they might be malformed
    except Exception:
        pass
    if df.empty:
        continue
    col_1 = df.iloc[:,0] #should be first column
    for item in col_1:
        #print("before: ", item)
        creditors.append(str(item))
    ok = cleanupCreditors(creditors) #cleanup creditors returns false if this is not a form 204
    #print("ok: ", ok)
    if not ok:
        #print("moving on!")
        continue
    #for item in creditors:
        #print("after: ", item)
    #for creditor in creditors:
        #print("creditor: ", creditor)
    col_last = df.iloc[:,-1]
    for item in col_last:
        if isinstance(item, str):
            nums = getNumsFromString(item)
            amounts.append(nums)
            #print("nums: ", nums)
    if caseID in forms:
        for creditor in creditors:
            if creditor != "":
                forms[caseID][0].append(creditor)
            
        for amountList in amounts:
            forms[caseID][1].append(amountList)
    else:
        forms[caseID] = (creditors, amounts)
    i += 1
print("all done ... again")

all done ... again


In [539]:
import json

with open('../data/final_forms.txt', 'w') as outfile:
    json.dump(forms, outfile)
print("done")

done
